In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import os
import scipy.io as sio
import numpy as np 

In [2]:
DATADIR = "./../image_data"
EEG_dir = "./../ImageNet-EEG"

CATEGORIES = ["animal", "blue_sky", "device","flying_object", "green_color", "musical_instrument","plant", "red_color", "vehicle","water", ]
subjects_EEG = ["1","2","3","4","5","6"] 

training_data = []
test_data = []



for category in CATEGORIES:  # do blue_sky and green_color
    path = os.path.join(DATADIR,category)  # create blue_sky and green_color
    class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=blue_sky 1=green_color
    img_counter = 0
    
    for img in os.listdir(path):  # iterate over each image blue_sky and green_color
        # print(img)
        img_counter +=1
        for subject in subjects_EEG: # do 1 ,2, 3, 4, 5, 6
            path_EEG = os.path.join(EEG_dir,subject)
            
            for eeg in os.listdir(path_EEG):
                if img == eeg.strip('.eeg.mat'):
                    if img_counter  < (np.array(os.listdir(path)).size*80/100): # 80 % training 
                        EEGs= sio.loadmat(os.path.join(path_EEG,eeg), matlab_compatible=True)
                        eeg_array = EEGs['x']
                        new_array = eeg_array[319:479,:]
                        training_data.append([new_array, class_num])  # add this to our training_date
                        # print("train"+eeg)
                    else:
                        EEGs= sio.loadmat(os.path.join(path_EEG,eeg), matlab_compatible=True)
                        eeg_array = EEGs['x']
                        new_array = eeg_array[319:479,:]
                        test_data.append([new_array, class_num])  # add this to our training_date
                        # print("test"+eeg)

                            
                                   

In [3]:
import random

random.shuffle(training_data)
random.shuffle(test_data)


In [4]:
for sample in training_data[:10]:
    print(sample[1])

0
0
5
9
8
0
5
6
1
8


In [5]:
for sample in test_data[:10]:
    print(sample[1])

9
0
1
1
7
6
6
2
6
5


In [6]:
X_train = []
y_train = []
for features,label in training_data:
    X_train.append(features)
    y_train.append(label)

X_test = []
y_test = []
for features,label in test_data:
    X_test.append(features)
    y_test.append(label)

In [7]:
X_train = np.array(X_train).reshape(-1, 160, 128)
X_train.shape

(1645, 160, 128)

In [8]:
X_test = np.array(X_test).reshape(-1, 160, 128)
X_test.shape

(471, 160, 128)

In [9]:
# my code starts here, Mahmoud 

In [10]:
from csv import reader
map_dict = dict() # dictionary to hold the target position of a channel in the target 2d map
# read csv file as a list of lists
with open('map_v2.csv', 'r',encoding='utf-8') as csv_file:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(csv_file)
    # Pass reader object to list() to get a list of lists
    m = list(csv_reader)
    print(m)
    for i,row in enumerate(m):
        for j,c in enumerate(row):
            # if the cell is not empty
            if c:
                c = c.replace(u'\ufeff','').strip() # handle encoding issue, to be enhanced
                assert(c not in map_dict)
                map_dict[c] = (i,j) # to be checked as it depends on the orientation
assert(len(map_dict)==128)
print(map_dict)


[['', '', '', 'Fp1', 'AFp1', 'AFz', 'AFp2', 'Fp2', '', '', ''], ['', 'AF7', 'AFF5h', 'AF3', 'AFF1h', '', 'AFF2h', 'AF4', 'AFF6h', 'AF8', ''], ['F9', 'F7', 'F5', 'F3', 'F1 ', 'Fz', 'F2', 'F4', 'F6', 'F8', 'F10'], ['FFT9h', 'FFT7h', 'FFC5h', 'FFC3h', 'FFC1h ', '', 'FFC2h ', 'FFC4h', 'FFC6h', 'FFT8h', 'FFT10h'], ['FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10'], ['FTT9h', 'FTT7h', 'FCC5h', 'FCC3h', 'FCC1h', '', 'FCC2h', 'FCC4h', 'FCC6h', 'FTT8h', 'FTT10h'], ['', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', ''], ['', 'TTP7h', 'CCP5h', 'CCP3h', 'CCP1h', '', 'CCP2h', 'CCP4h', 'CCP6h', 'TTP8h', ''], ['TP9', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10'], ['TPP9h', 'TPP7h', 'CPP5h', 'CPP3h', 'CPP1h ', '', 'CPP2h ', 'CPP4h', 'CPP6h', 'TPP8h', 'TPP10h'], ['P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10'], ['PPO9h', 'PO7', 'PPO5h', 'PO3', 'PPO1h ', 'POz', 'PPO2h ', 'PO4', 'PPO6h', 'PO8', 'PPO10h'], ['PO9', 'POO9h', '',

In [11]:
map_dict['FFT9h']

(3, 0)

In [12]:
# read csv file as a list of lists
with open('key.csv', 'r',encoding='utf-8') as csv_file:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(csv_file)
    # Pass reader object to list() to get a list of lists
    key = list(csv_reader)
    key =[item for sublist in key for item in sublist] # flatten the key
    assert(len(key)==128)
    print(key)

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz', 'F9', 'AFF1h', 'FFC1h', 'FFC5h', 'FTT7h', 'FCC3h', 'CCP1h', 'CCP5h', 'TPP7h', 'P9', 'PPO9h', 'PO9', 'O9', 'OI1h', 'PPO1h', 'CPP3h', 'CPP4h', 'PPO2h', 'OI2h', 'O10', 'PO10', 'PPO10h', 'P10', 'TPP8h', 'CCP6h', 'CCP2h', 'FCC4h', 'FTT8h', 'FFC6h', 'FFC2h', 'AFF2h', 'F10', 'AFp1', 'AFF5h', 'FFT9h', 'FFT7h', 'FFC3h', 'FCC1h', 'FCC5h', 'FTT9h', 'TTP7h', 'CCP3h', 'CPP1h', 'CPP5h', 'TPP9h', 'POO9h', 'PPO5h', 'POO1', 'POO2', 'PPO6h', 'POO10h', 'TPP10h', 'CPP6h', 'CPP2h', 'CCP4h', 'TTP8h', 'FTT10h', 'FCC6h', 'FCC2h', 'FFC4h', 'FFT8h', 'FFT10h', 'AFF6h', 'AFp2']


In [13]:
#check if all channels are mapped
for k in key:
    if k not in map_dict:
        print(k)

In [14]:
def vector_to_map(vector):
    #print(vector.shape)
    #print(vector)
    assert(vector.shape==(128,))
    m = np.zeros((14,11))
    for idx, value in np.ndenumerate(vector):
        assert(key[idx[0]] in map_dict) # np return index as tuple 
        x,y = map_dict[key[idx[0]]]
        m[x,y] = value
        #print(m[x,y])
    assert(np.count_nonzero(m)==128)
    return m

In [15]:
X_train_encoded = np.zeros((X_train.shape[0],X_train.shape[1], 14, 11))
X_test_encoded = np.zeros((X_test.shape[0],X_test.shape[1], 14, 11))

In [16]:
vector_to_map(X_train[0,0])

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.10830985e-01,  2.19863560e-02, -6.17885590e-01,
        -1.57540739e+00,  2.76462078e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.08009958e+00,  2.58832335e-01,
        -8.92534554e-02,  2.32074678e-01,  0.00000000e+00,
         1.40283450e-01,  1.51338029e+00,  1.39023438e-01,
         6.91904724e-01,  0.00000000e+00],
       [ 1.48089156e-01, -9.31525677e-02, -1.27381063e+00,
         1.04413241e-01, -5.05397141e-01,  2.81015933e-01,
         7.04977095e-01,  2.50031739e-01,  3.24029773e-01,
         2.31906548e-01,  2.90393442e-01],
       [ 1.80653679e+00,  4.16779488e-01,  2.19007587e+00,
        -7.52619728e-02,  3.08259845e+00,  0.00000000e+00,
         7.60245740e-01,  3.88766855e-01, -2.05961112e-02,
         6.58940449e-02,  1.39102840e+00],
       [-5.23829758e-02, -5.58087463e-03,  2.43625566e-01,
        -1.12319577e+00,  5.48436046e-01, -1.41247241e-02,
  

In [17]:
# can this code be enhanced? 
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        X_train_encoded[i,j] = vector_to_map(X_train[i,j])
        
for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        X_test_encoded[i,j] = vector_to_map(X_test[i,j])

In [18]:
print( X_train_encoded.shape)
print( X_test_encoded.shape)

(1645, 160, 14, 11)
(471, 160, 14, 11)


In [19]:
import pickle

pickle_out = open("X_train_encoded_full_2D.pickle","wb")
pickle.dump(X_train_encoded, pickle_out,  protocol=4)
pickle_out.close()

pickle_out = open("y_train_full.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

# We can always load it in to our current script, or a totally new one by doing:

''' 
pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

'''

pickle_out = open("X_test_encoded_full_2D.pickle","wb")
pickle.dump(X_test_encoded, pickle_out, protocol=4)
pickle_out.close()

pickle_out = open("y_test_full.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()

In [20]:
X_train_encoded[1,1,5:10,4:10]

array([[-0.54057133,  0.        , -0.07066333, -0.05338725, -1.4753871 ,
        -0.10223411],
       [-0.41411111,  0.20282668,  0.0695984 ,  0.09174069, -0.04449355,
         0.36181793],
       [ 0.06166207,  0.        , -0.91785163, -0.10635728, -0.46212474,
        -0.05435545],
       [ 0.11871256, -0.12479617,  0.1672723 , -0.43977579,  0.28234372,
        -0.53334415],
       [-0.05139366,  0.        ,  0.03602599,  0.12663788,  0.84155399,
        -0.05293268]])